In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager()

/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 107 runs


In [3]:
M.show()

  0 - 3dshapes-fdvae_0077-5920872-49_191213-201526
  1 - 3dshapes-wae_0077-5920872-11_191213-201525
  2 - 3dscr-dvae_0078-5920873-7_191213-202209
  3 - 3dscr-fdvae_0078-5920873-9_191213-202210
  4 - gan_0070-5920322-14_191213-093204
  5 - 3dshapes-fvae_0077-5920872-38_191213-201526
  6 - 3dshapes-fdwae_0077-5920872-46_191213-201528
  7 - fwae-celeba_0072-5920866-2_191213-200021
  8 - gan_0070-5920322-13_191213-093204
  9 - 3dshapes-wgan_0080-5920891-3_191214-090740
 10 - 3dshapes-wpp_0077-5920872-24_191213-201524
 11 - vae-celeba_0072-5920866-1_191213-200022
 12 - fvae-celeba_0072-5920866-3_191213-200021
 13 - 3dshapes-fdwae_0077-5920872-48_191213-201525
 14 - fdwae-celeba_0072-5920866-8_191213-200021
 15 - fdvae-celeba_0074-5920868-4_191213-200516
 16 - celeba-dwae_0076-5920870-6_191213-201051
 17 - 3dshapes-wgan_0077-5920872-0_191213-201539
 18 - 3dshapes-wgan_0080-5920891-2_191214-090739
 19 - 3dshapes-wpp_0077-5920872-31_191213-201529
 20 - 3dshapes-fvae_0077-5920872-39_191213-2015

In [4]:
run = M[21]
run.keys()

odict_keys(['name', 'path', 'ckpt_path', 'config'])

In [5]:
train.render_dict(run.config)

In [6]:
d, ad = M.compare(17, 18, bi=True)
train.render_dict(d)

In [7]:
train.render_dict(ad)

In [8]:
M[17].config['model.disc_steps']

1

In [9]:
M[17].config['model','disc_steps']

1

In [11]:
import yaml

In [12]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

'/is/ei/fleeb/workspace/chome/trained_nets'

In [15]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

3dshapes-fdvae_0077-5920872-49_191213-201526 fixed
3dshapes-wae_0077-5920872-11_191213-201525 fixed
3dscr-dvae_0078-5920873-7_191213-202209 fixed
3dscr-fdvae_0078-5920873-9_191213-202210 fixed
gan_0070-5920322-14_191213-093204 

/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


fixed
3dshapes-fvae_0077-5920872-38_191213-201526 fixed
3dshapes-fdwae_0077-5920872-46_191213-201528 fixed
fwae-celeba_0072-5920866-2_191213-200021 fixed
gan_0070-5920322-13_191213-093204 fixed
3dshapes-wgan_0080-5920891-3_191214-090740 fixed
3dshapes-wpp_0077-5920872-24_191213-201524 fixed
vae-celeba_0072-5920866-1_191213-200022 fixed
fvae-celeba_0072-5920866-3_191213-200021 fixed
3dshapes-fdwae_0077-5920872-48_191213-201525 fixed
fdwae-celeba_0072-5920866-8_191213-200021 fixed
fdvae-celeba_0074-5920868-4_191213-200516 fixed
celeba-dwae_0076-5920870-6_191213-201051 fixed
3dshapes-wgan_0077-5920872-0_191213-201539 fixed
3dshapes-wgan_0080-5920891-2_191214-090739 fixed
3dshapes-wpp_0077-5920872-31_191213-201529 fixed
3dshapes-fvae_0077-5920872-39_191213-201528 fixed
3dshapes-vae_0077-5920872-22_191213-201525 fixed
3dscr-fvae_0078-5920873-3_191213-202212 fixed
celeba-wae_0076-5920870-0_191213-201050 fixed
3dshapes-wae_0077-5920872-14_191213-201548 fixed
dvae-celeba_0073-5920867-2_191213-

In [16]:
data['parents']

KeyError: 'parents'